Background:
A new strain of ransomware has managed to penetrate several critical government networks and NSA has been called upon to assist in remediating the infection to prevent massive data losses. For each infected machine, an encrypted copy of the key needed to decrypt the ransomed files has been stored in a smart contract on the Ethereum blockchain* and is set to only be unlocked upon receipt of the ransom payment. Your mission is to ultimately (1) find a way to unlock the ransomware without giving in to the attacker’s demands and (2) figure out a way to recover all of the funds already paid by other victims. Are YOU up to the challenge?


* - for the purposes of this challenge, a private blockchain has been created with no real monetary value associated with the Ether. See http://www.ethdocs.org/en/latest/ for more information about Ethereum.



# Task 0
We have acquired a packet capture (PCAP) file showing network traffic between a newly infected victim computer and the attacker's listening post (LP). The LP is simply a server that is listening for incoming connections on a certain port. To get started, analyze the network capture and submit the IP address of the attacker's LP.


Downloads:
Task0/register.pcap

Thoughts behind answer:
Figure out how to open pcap file. Either wireshark or scapy would do the job. We see two ips 172.26.175.126 10.108.159.179. Since 172.26.175.126 received the most traffic. I assume this is the attacker. 

In [14]:
import  scapy.all  as s
data_file = "./Task0/register.pcap"
stats = s.rdpcap(data_file).res
total_receiver = {}
for r in stats:
    if r.payload.dst not in total_receiver:
        total_receiver[r.payload.dst] = 0
    total_receiver[r.payload.dst] =  total_receiver[r.payload.dst] + r.payload.len
    
print(total_receiver)
total_sender = {}

for r in stats:
    if r.payload.src not in total_sender:
        total_sender[r.payload.src] = 0
    total_sender[r.payload.src] =  total_sender[r.payload.src] + r.payload.len
print(total_sender)

{'10.108.159.179': 1856, '172.26.175.126': 2728}
{'10.108.159.179': 2728, '172.26.175.126': 1856}


# Task 1 
Whenever the ransomware infects a new victim, it calls out to the attacker LP with (1) a unique victim identifier, (2) encrypted copy of the key used to ransom victim files, and (3) a one-time passcode (OTP) used for authentication. The LP, in turn, connects to an Ethereum client and deploys a Ransom "smart" contract on the Ethereum blockchain that authenticates with other parts of the attacker's infrastructure. After authentication, the LP returns back to the ransomware (running on victim computer) information describing where the victim should send funds. Ransom payment is made to an Escrow contract, so the LP responds with the Escrow contract's blockchain address, a 20-byte value derived from a private key that corresponds to a unique account/contract on the blockchain. Ransom and Escrow smart contracts and attacker infrastructure will be explained more fully in subsequent tasks.


The ransomware removes itself from victim computers after completing infection and registration processes. Therefore, a complete ransomware executable hasn't yet been found. The only binaries it leaves behind are two shared libraries - libclient_comms.so and libclient_crypt.so. It's unclear if leaving these binaries on disk is an oversight or intentional. Your task is to analyze the ransomware client-registration communications protocol and submit the information requested below. To assist you, we have provided the two above-mentioned shared library binaries and a packet capture (pcap) file showing registration-related network traffic between victim computer and attacker LP. The library binaries were taken from the same victim computer shown in the pcap.


Please submit the following:

Victim Identifier
Encrypted Ransom Key
One-Time Passcode (OTP) used to authenticate the client to the ransomware LP
Escrow contract address

Downloads:

Network Traffic PCAP (register.pcap)
Malware communications library (libclient_comms.so)
Malware cryptography library (libclient_crypt.so)
Ransom Note (ransom_note.txt)

In [1]:
! cat ./Task1/ransom_note.txt

In [2]:
! objdump -D ./Task1/libclient_crypt.so


./Task1/libclient_crypt.so:	file format ELF64-x86-64

Disassembly of section .note.gnu.build-id:
.note.gnu.build-id:
     1c8:	04 00 	addb	$0, %al
     1ca:	00 00 	addb	%al, (%rax)
     1cc:	14 00 	adcb	$0, %al
     1ce:	00 00 	addb	%al, (%rax)
     1d0:	03 00 	addl	(%rax), %eax
     1d2:	00 00 	addb	%al, (%rax)
     1d4:	47 4e  <unknown>
     1d6:	55 	pushq	%rbp
     1d7:	00 81 d5 fb d0 74 	addb	%al, 1959853013(%rcx)
     1dd:	65 c8 aa ba 11 	enter	$-17750, $17
     1e2:	c2 2f 86 	retq	$-31185
     1e5:	73 e2 	jae	-30 <.note.gnu.build-id+0x1>
     1e7:	ce  <unknown>
     1e8:	b3 8e 	movb	$-114, %bl
     1ea:	01 3f 	addl	%edi, (%rdi)
Disassembly of section .gnu.hash:
.gnu.hash:
     1f0:	03 00 	addl	(%rax), %eax
     1f2:	00 00 	addb	%al, (%rax)
     1f4:	1a 00 	sbbb	(%rax), %al
     1f6:	00 00 	addb	%al, (%rax)
     1f8:	02 00 	addb	(%rax), %al
     1fa:	00 00 	addb	%al, (%rax)
     1fc:	07  <unknown>
     1fd:	00 00 	addb	%al, (%rax)
     1ff:	00 80 11 14 08 00 	addb	%al, 529425(%ra

     94e:	00 00 	addb	%al, (%rax)
     950:	08 00 	orb	%al, (%rax)
     952:	00 00 	addb	%al, (%rax)
     954:	00 00 	addb	%al, (%rax)
     956:	00 00 	addb	%al, (%rax)
     958:	e0 30 	loopne	48
     95a:	20 00 	andb	%al, (%rax)
     95c:	00 00 	addb	%al, (%rax)
     95e:	00 00 	addb	%al, (%rax)
     960:	d0 2f 	shrb	(%rdi)
     962:	20 00 	andb	%al, (%rax)
     964:	00 00 	addb	%al, (%rax)
     966:	00 00 	addb	%al, (%rax)
     968:	06  <unknown>
     969:	00 00 	addb	%al, (%rax)
     96b:	00 04 00 	addb	%al, (%rax,%rax)
     96e:	00 00 	addb	%al, (%rax)
     970:	00 00 	addb	%al, (%rax)
     972:	00 00 	addb	%al, (%rax)
     974:	00 00 	addb	%al, (%rax)
     976:	00 00 	addb	%al, (%rax)
     978:	d8 2f 	fsubrs	(%rdi)
     97a:	20 00 	andb	%al, (%rax)
     97c:	00 00 	addb	%al, (%rax)
     97e:	00 00 	addb	%al, (%rax)
     980:	06  <unknown>
     981:	00 00 	addb	%al, (%rax)
     983:	00 05 00 00 00 00 	addb	%al, (%rip)
     989:	00 00 	addb	%al, (%rax)
     98b:	00 00 	addb	%al, (%r

    228e:	00 00 	addb	%al, (%rax)
    2290:	b0 f2 	movb	$-14, %al
    2292:	ff ff  <unknown>
    2294:	92 	xchgl	%edx, %eax
    2295:	01 00 	addl	%eax, (%rax)
    2297:	00 00 	addb	%al, (%rax)
    2299:	42 0e  <unknown>
    229b:	10 8e 02 42 0e 18 	adcb	%cl, 403587586(%rsi)
    22a1:	8d 03 	leal	(%rbx), %eax
    22a3:	45 0e  <unknown>
    22a5:	20 8c 04 41 0e 28 86 	andb	%cl, -2044195263(%rsp,%rax)
    22ac:	05 44 0e 30 83 	addl	$2200964676, %eax
    22b1:	06  <unknown>
    22b2:	4d 0e  <unknown>
    22b4:	60  <unknown>
    22b5:	02 e4 	addb	%ah, %ah
    22b7:	0a 0e 	orb	(%rsi), %cl
    22b9:	30 41 0e 	xorb	%al, 14(%rcx)
    22bc:	28 41 0e 	subb	%al, 14(%rcx)
    22bf:	20 42 0e 	andb	%al, 14(%rdx)
    22c2:	18 42 0e 	sbbb	%al, 14(%rdx)
    22c5:	10 42 0e 	adcb	%al, 14(%rdx)
    22c8:	08 49 0b 	orb	%cl, 11(%rcx)
    22cb:	00 00 	addb	%al, (%rax)
    22cd:	00 00 	addb	%al, (%rax)
    22cf:	00 2c 00 	addb	%ch, (%rax,%rax)
    22d2:	00 00 	addb	%al, (%rax)
    22d4:	7c 01 	jl	1 <.eh_frame+

In [3]:
! objdump -D ./Task1/libclient_comms.so


./Task1/libclient_comms.so:	file format ELF64-x86-64

Disassembly of section .note.gnu.build-id:
.note.gnu.build-id:
     1c8:	04 00 	addb	$0, %al
     1ca:	00 00 	addb	%al, (%rax)
     1cc:	14 00 	adcb	$0, %al
     1ce:	00 00 	addb	%al, (%rax)
     1d0:	03 00 	addl	(%rax), %eax
     1d2:	00 00 	addb	%al, (%rax)
     1d4:	47 4e  <unknown>
     1d6:	55 	pushq	%rbp
     1d7:	00 31 	addb	%dh, (%rcx)
     1d9:	56 	pushq	%rsi
     1da:	1b 23 	sbbl	(%rbx), %esp
     1dc:	ac 	lodsb	(%rsi), %al
     1dd:	22 fb 	andb	%bl, %bh
     1df:	0c 33 	orb	$51, %al
     1e1:	a8 b8 	testb	$-72, %al
     1e3:	1d 07 6d 67 05 	sbbl	$90664199, %eax
     1e8:	42 06  <unknown>
     1ea:	15  <unknown>
     1eb:	f8 	clc
Disassembly of section .gnu.hash:
.gnu.hash:
     1f0:	03 00 	addl	(%rax), %eax
     1f2:	00 00 	addb	%al, (%rax)
     1f4:	15 00 00 00 01 	adcl	$16777216, %eax
     1f9:	00 00 	addb	%al, (%rax)
     1fb:	00 06 	addb	%al, (%rsi)
     1fd:	00 00 	addb	%al, (%rax)
     1ff:	00 88 c0 22 09 00 	addb	

    10bd:	49 89 45 18 	movq	%rax, 24(%r13)
    10c1:	49 8b 47 20 	movq	32(%r15), %rax
    10c5:	49 89 45 20 	movq	%rax, 32(%r13)
    10c9:	49 8b 47 28 	movq	40(%r15), %rax
    10cd:	49 89 45 28 	movq	%rax, 40(%r13)
    10d1:	49 8b 47 30 	movq	48(%r15), %rax
    10d5:	49 89 45 30 	movq	%rax, 48(%r13)
    10d9:	49 8b 47 38 	movq	56(%r15), %rax
    10dd:	49 89 45 38 	movq	%rax, 56(%r13)
    10e1:	48 8d 83 60 03 00 00 	leaq	864(%rbx), %rax
    10e8:	48 89 c7 	movq	%rax, %rdi
    10eb:	f3 48 a5 	rep		movsq	(%rsi), %es:(%rdi)
    10ee:	48 89 df 	movq	%rbx, %rdi
    10f1:	e8 3a fa ff ff 	callq	-1478 <.text+0x100>
    10f6:	85 c0 	testl	%eax, %eax
    10f8:	0f 85 e2 fd ff ff 	jne	-542 <start_client+0x330>
    10fe:	48 8b 7c 24 20 	movq	32(%rsp), %rdi
    1103:	4c 8b 6c 24 20 	movq	32(%rsp), %r13
    1108:	4c 89 f0 	movq	%r14, %rax
    110b:	b9 60 00 00 00 	movl	$96, %ecx
    1110:	48 c7 84 24 88 06 00 00 90 02 00 00 	movq	$656, 1672(%rsp)
    111c:	48 c7 84 24 80 06 00 00 00 00 00 00 	movq	$0,